In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC CLASSIFIER USING MAMMOGRAM PATCHES IN BCDR
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v003'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.xception import Xception
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [4]:
len(model.layers)

139

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class01/train01', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class01/valid01', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 532 images belonging to 2 classes.
Found 134 images belonging to 2 classes.


In [7]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_xception_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
66/66 [==============================] - 79s 292ms/step - loss: 0.6126 - accuracy: 0.7290 - val_loss: 0.3933 - val_accuracy: 0.8203


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/3
66/66 [==============================] - 11s 171ms/step - loss: 0.4830 - accuracy: 0.7920 - val_loss: 0.3493 - val_accuracy: 0.8438
Epoch 3/3
66/66 [==============================] - 11s 173ms/step - loss: 0.4453 - accuracy: 0.7996 - val_loss: 0.3965 - val_accuracy: 0.8281


In [9]:
#Training stage 2
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
66/66 [==============================] - 23s 188ms/step - loss: 0.4752 - accuracy: 0.7939 - val_loss: 0.3270 - val_accuracy: 0.8594
Epoch 2/10
66/66 [==============================] - 11s 172ms/step - loss: 0.4263 - accuracy: 0.8168 - val_loss: 0.2827 - val_accuracy: 0.8828
Epoch 3/10
66/66 [==============================] - 11s 167ms/step - loss: 0.4257 - accuracy: 0.8168 - val_loss: 0.2913 - val_accuracy: 0.8750
Epoch 4/10
66/66 [==============================] - 10s 158ms/step - loss: 0.3897 - accuracy: 0.8473 - val_loss: 0.3122 - val_accuracy: 0.8750
Epoch 5/10
66/66 [==============================] - 11s 167ms/step - loss: 0.3887 - accuracy: 0.8340 - val_loss: 0.2663 - val_accuracy: 0.8828
Epoch 6/10
66/66 [==============================] - 11s 165ms/step - loss: 0.3914 - accuracy: 0.8244 - val_loss: 0.2804 - val_accuracy: 0.8906
Epoch 7/10
66/66 [==============================] - 11s 165ms/step - loss: 0.3362 - accuracy: 0.8607 - val_loss: 0.2306 - val_accuracy: 0.8984

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150
66/66 [==============================] - 23s 208ms/step - loss: 0.3932 - accuracy: 0.8473 - val_loss: 0.3310 - val_accuracy: 0.8359
Epoch 2/150
66/66 [==============================] - 12s 176ms/step - loss: 0.3449 - accuracy: 0.8550 - val_loss: 0.3835 - val_accuracy: 0.8281
Epoch 3/150
66/66 [==============================] - 12s 176ms/step - loss: 0.3729 - accuracy: 0.8454 - val_loss: 0.3846 - val_accuracy: 0.8359
Epoch 4/150
66/66 [==============================] - 11s 170ms/step - loss: 0.3554 - accuracy: 0.8435 - val_loss: 0.3686 - val_accuracy: 0.8594
Epoch 5/150
66/66 [==============================] - 11s 166ms/step - loss: 0.3490 - accuracy: 0.8683 - val_loss: 0.3628 - val_accuracy: 0.8438
Epoch 6/150
66/66 [==============================] - 12s 173ms/step - loss: 0.3836 - accuracy: 0.8473 - val_loss: 0.3515 - val_accuracy: 0.8750
Epoch 7/150
66/66 [==============================] - 12s 176ms/step - loss: 0.3489 - accuracy: 0.8683 - val_loss: 0.3489 - val_accuracy:

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class01/valid01', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)


Found 134 images belonging to 2 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[86  5]
 [10 33]]
Accuracy: 0.8880597014925373
ROC AUC score: 0.9511883465371838
Precision score: 0.868421052631579
Recall score: 0.7674418604651163
F1 score: 0.8148148148148148
MCC score: 0.7379219529223237


In [13]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[7.39937462e-03]
 [2.58561552e-01]
 [7.66102150e-02]
 [3.08798882e-03]
 [2.71666422e-02]
 [9.70395803e-01]
 [1.68538000e-02]
 [8.93450081e-01]
 [1.91656381e-01]
 [2.09485944e-02]
 [4.86025624e-02]
 [9.07115489e-02]
 [7.52829731e-01]
 [2.86584999e-02]
 [4.52972561e-01]
 [2.85991520e-01]
 [6.31036833e-02]
 [4.47082639e-01]
 [2.79729635e-01]
 [2.71889597e-01]
 [4.56672832e-02]
 [3.03211093e-01]
 [1.68396547e-01]
 [2.36343503e-01]
 [1.56900287e-01]
 [4.73681748e-01]
 [9.99300405e-02]
 [9.24902130e-03]
 [7.33546540e-02]
 [1.57900006e-01]
 [5.45532294e-02]
 [4.81011629e-01]
 [3.51369560e-01]
 [1.14060029e-01]
 [9.86300558e-02]
 [8.20030924e-04]
 [2.84904897e-01]
 [1.30156219e-01]
 [2.55890638e-02]
 [1.26719639e-01]
 [4.26105075e-02]
 [3.74300361e-01]
 [9.12248269e-02]
 [4.74836044e-02]
 [4.04429762e-03]
 [3.04551095e-01]
 [3.74184161e-01]
 [3.17841172e-01]
 [2.61924751e-02]
 [8.01773518e-02]
 [1.23427138e-01]
 [1.32012680e-01]
 [1.27710357e-01]
 [1.68639705e-01]
 [2.11287901e-01]
 [3.384680

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class01/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)

Found 169 images belonging to 2 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[106   7]
 [ 19  37]]
Accuracy: 0.8461538461538461
ROC AUC score: 0.8847977243994942
Precision score: 0.8409090909090909
Recall score: 0.6607142857142857
F1 score: 0.74
MCC score: 0.6422587434433877


In [16]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[2.67536849e-01]
 [2.76918381e-01]
 [1.44917637e-01]
 [8.35792199e-02]
 [2.89492533e-02]
 [7.07333849e-04]
 [2.79785275e-01]
 [8.06932151e-02]
 [2.30950285e-02]
 [2.36752201e-02]
 [2.02550087e-02]
 [3.96364741e-02]
 [2.38597631e-01]
 [1.42256305e-01]
 [7.11956695e-02]
 [8.02369788e-03]
 [5.99794805e-01]
 [4.29255188e-01]
 [4.86733438e-03]
 [6.21412456e-01]
 [5.20603538e-01]
 [5.42491563e-02]
 [7.33607486e-02]
 [2.00655982e-01]
 [2.91766622e-03]
 [4.73926496e-03]
 [1.82158947e-02]
 [1.46861121e-01]
 [7.46897042e-01]
 [3.24303657e-01]
 [1.41758565e-02]
 [3.81125450e-01]
 [6.59660921e-02]
 [2.48876691e-01]
 [1.53300971e-01]
 [5.55073051e-03]
 [1.92550480e-01]
 [4.29072082e-01]
 [3.02379429e-02]
 [5.26693106e-01]
 [1.62382439e-01]
 [3.19977164e-01]
 [3.28021124e-02]
 [3.99377644e-02]
 [8.81829336e-02]
 [2.33648885e-02]
 [4.78509031e-02]
 [1.01034939e-02]
 [4.37066555e-01]
 [4.38937172e-02]
 [3.21982019e-02]
 [3.23874712e-01]
 [4.90927249e-02]
 [7.12789502e-03]
 [2.61877514e-02]
 [4.288154